In [7]:
import luigi
import pandas as pd
import os
import logging



In [8]:
class ReadAllData(luigi.Task):
    def run(self):
        df_delinquance = pd.read_csv("./CleanData/delinquance.csv", sep=",")
        df_canton = pd.read_csv("./CleanData/canton.csv", sep=",")
        df_chomeurs = pd.read_csv("./CleanData/chomeurs.csv", sep=",")

        
        merge_data = pd.merge(df_canton, df_delinquance, on='Code du departement', how='inner')
        merge_data = pd.merge(merge_data, df_chomeurs, on='Code du departement', how='inner')

        merge_data.to_csv(self.output().path, index=False)

    def output(self):
        return luigi.LocalTarget('./CleanData/data.csv')

In [9]:

config = luigi.configuration.get_config()
config.set('core', 'no_lock', 'False')


luigi.build([ReadAllData()], local_scheduler=False, no_lock=True)

DEBUG: Checking if ReadAllData() is complete
INFO: Informed scheduler that task   ReadAllData__99914b932b   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 10432] Worker Worker(salt=1811937867, workers=1, host=DESKTOP-PBH195U, username=Marin, pid=10432) running   ReadAllData()
INFO: [pid 10432] Worker Worker(salt=1811937867, workers=1, host=DESKTOP-PBH195U, username=Marin, pid=10432) done      ReadAllData()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ReadAllData__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=1811937867, workers=1, host=DESKTOP-PBH195U, username=Marin, pid=10432) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 ran successfully:
    - 1

True